Load impurities data

In [21]:
import pandas as pd
import yaml

# change this path for other impurities
with open("config\\METHANE_IMPURITIES.yaml", 'r') as config_file:
    config = yaml.safe_load(config_file)

BASE_COMPOSITIONS_LOWER_LIMITS = pd.Series(
    config['BASE_COMPOSITIONS_LOWER_LIMITS'], 
    name="Base Compositions [vol/vol]"
)
IMPURITIES_UPPER_LIMITS = pd.Series(
    config['IMPURITIES_UPPER_LIMITS'],
    name="Impurities Upper Limits [vol/vol]"
)

Define a domain for temperature and pressure, 

$T\in[T_{min}, T_{max}]\quad p\in[P_{min}, P_{max}]$

In [22]:
from impurities import generate_temperature_pressure_samples

doe = generate_temperature_pressure_samples(
    config["DOE"]["n_grid_temperature"],
    config["DOE"]["n_grid_pressure"],
    config["DOE"]["temperature_range"],
    config["DOE"]["pressure_range"],
)

Get reference fluid property data for the measured component. In this case, single phase methane

In [23]:
from generate_data import get_fluid_property

df_ref = get_fluid_property(
    doe=doe,
    hFld=";".join(
        BASE_COMPOSITIONS_LOWER_LIMITS.add_prefix("refprop/FLUIDS/").index
        ),
    hOut=config['FLUID_PROPERTY'],
    z=list(BASE_COMPOSITIONS_LOWER_LIMITS.values) + \
        [0.0] * (20 - len(BASE_COMPOSITIONS_LOWER_LIMITS)), 
)
df_ref.describe()

,D,T,P
count,100.000000,100.000000,1.000000e+02
mean,6.693920,316.650000,1.076325e+06
std,4.128864,4.169752,6.575378e+05
min,0.306657,310.150000,5.132500e+04
25%,3.113968,313.038889,5.068806e+05
50%,6.644276,316.650000,1.076325e+06
75%,10.244433,320.261111,1.645769e+06
max,13.487231,323.150000,2.101325e+06


Get reference fluid property data for each of the impurities.

In [24]:
from impurities import get_impurities_properties

df_impurities = get_impurities_properties(
    fluids=list(IMPURITIES_UPPER_LIMITS.keys()),
    doe=doe,
    df_ref=df_ref,
    fluid_property=config['FLUID_PROPERTY'],
)

Calculate a metric to quantify the error over the range.
This is done by intergating the density error over the operating range.

Mathematically a double integral of the fluid property error, $\int_T\int_p (\hat\phi-\phi_{ref})dpdT$

In [25]:
from impurities import get_impact_metrics

impact_metrics = {config['FLUID_PROPERTY']: get_impact_metrics(df_impurities, config['FLUID_PROPERTY'])}
impact_metrics[config['FLUID_PROPERTY']]

WATER       2.623746e+10
CO2         3.321677e+08
ETHANE      1.839123e+08
OXYGEN      1.726640e+08
ETHYLENE    1.503632e+08
NITROGEN    1.271424e+08
Name: Integral of D Deviation over Temperature and Pressure Range, dtype: float64

And then sort these impact metrics on most significant to least significant

In [26]:
impurities_sorted = list(impact_metrics[config['FLUID_PROPERTY']].sort_values(ascending=False).index)
impurities_sorted

['WATER', 'CO2', 'ETHANE', 'OXYGEN', 'ETHYLENE', 'NITROGEN']

We then need to normalise the original compositions so that the total impurities and base compositions sum to 100%

In [27]:
"""BRUTE FORCE METHOD"""
total_impurities = (1 - BASE_COMPOSITIONS_LOWER_LIMITS.sum())
sum_impurities = 0
impurities_sorted_normalized = {}
for impurity in impurities_sorted:
    sum_impurities += IMPURITIES_UPPER_LIMITS[impurity]
    if sum_impurities < total_impurities:
        impurities_sorted_normalized[impurity] = IMPURITIES_UPPER_LIMITS[impurity]
    else:
        impurities_sorted_normalized[impurity] = IMPURITIES_UPPER_LIMITS[impurity] - (sum_impurities - total_impurities)
        break

compositions = pd.concat([BASE_COMPOSITIONS_LOWER_LIMITS, pd.Series(impurities_sorted_normalized)])
print(compositions.sum())
compositions

1.0


METHANE     0.99500
WATER       0.00002
CO2         0.00010
ETHANE      0.00060
OXYGEN      0.00010
ETHYLENE    0.00200
NITROGEN    0.00218
dtype: float64

Hence we now have a total compositions array in vol/vol or mol/mol depending on the spec. 

To convert to mol/mol we can calculate the density of each component and total molar volume.

We known the filling conditions for the gas bottles (where the vol/vol compositions array is given for) temperature, $T_{filling}$ and filling pressure, $p_{filling}$ so can calculate the filling density, $\rho_{filling}=f(T_{filling}, p_{filling})$.

Hence we can calculate the molar density, $\rho_M=\rho / M$ where $M$ is the molar mass of the component.

We can then calculate the number of moles per unit volume using the volume compositions of each component divided by the molar density, $x_{V,i}/\rho_M$

We can then calculate the molar composition, $x_{m,i}$ for each component, $i$, as the ratio of the number of moles of the component, $n_i$, to the total number of moles for each component in a control volume, $V$,

$$x_{m,i}=\frac{n_i}{\sum_i n_i}=\frac{n_i / V}{1/V \sum_i n_i}=\frac{x_{V,i}/\rho_M}{\sum_i x_{V,i}/\rho_M}

In [28]:
from refprop.refprop import RefpropInterface

def convert_compositions_to_moles_unit_volume(compositions, temperature_filling, pressure_filling):
    def get_molar_mass(fluid):
        refprop = RefpropInterface(r"T:\Joseph McGovern\Code\GitHub\refprop-dotnet\refprop", fluid)
        refprop.setup_refprop()
        M = refprop.fluid_info['Molar Mass [g/mol]'] / 1000  # kg/mol
        return M

    def get_property(fluid, fluid_property, temperature, pressure):
        return get_fluid_property(
            pd.DataFrame({"Temperature [K]": [temperature], "Pressure [Pa]": [pressure]}, index=[0]), 
            f"refprop/FLUIDS/{fluid}",
            fluid_property, 
            [1.0] + [0] * 19
        )[fluid_property].values[0]  # kg/m3

    def get_moles_unit_volume(x_v, M, rho):
        rho_m = rho / M
        return x_v * rho_m

    df_filling = pd.DataFrame()
    for fluid in compositions.index:
        rho = get_property(
            fluid, "D", temperature_filling, pressure_filling
        )  # kg/m3
        M = get_molar_mass(fluid=fluid)  # kg/mol
        n = get_moles_unit_volume(compositions[fluid], M, rho)
        df_filling = pd.concat([
            df_filling,
            pd.DataFrame({
                "Filling Temperature [K]": [temperature_filling],
                "Filling Pressure [Pa]": [pressure_filling],
                "Filling Density [kg/m3]": [rho],
                "Molar Mass [kg/mol]": [M],
                "Moles Unit Volume [mol/L]": [n],
            }, index=[fluid])
        ])
    df_filling["Composition [mol/mol]"] = df_filling["Moles Unit Volume [mol/L]"] / df_filling["Moles Unit Volume [mol/L]"].sum()
    df_filling["Composition [L/L]"] = compositions.values
    return df_filling

if config['COMPOSITIONS_BASIS'] == "volume":
    compositions_worst_case = convert_compositions_to_moles_unit_volume(
        compositions=compositions, 
        temperature_filling=config['FILLING_CONDITIONS']['TEMPERATURE'], 
        pressure_filling=config['FILLING_CONDITIONS']['PRESSURE']
    )["Composition [mol/mol]"]
elif config['COMPOSITIONS_BASIS'] == "mole":
    compositions_worst_case = compositions
else:
    raise Exception("Invalid 'COMPOSITIONS_BASIS'")

Since we now have a "worst-case" compositions array, we can calculate the fluid property for this and a "worst-case" error

In [29]:
df_worst_case = get_fluid_property(
    doe=doe,
    hFld=";".join(compositions_worst_case.add_prefix("refprop/FLUIDS/").index),
    hOut=config['FLUID_PROPERTY'],
    z=list(compositions_worst_case.values) + \
        [0.0] * (20 - compositions_worst_case.shape[0]),
)

deviation_col = f"{config['FLUID_PROPERTY']} Deviation [%]"
df_worst_case[deviation_col] = (df_worst_case.D - df_ref.D) / df_ref.D * 100.0
df_worst_case.describe()

,D,T,P,D Deviation [%]
count,100.000000,100.000000,1.000000e+02,100.000000
mean,6.725107,316.650000,1.076325e+06,0.463062
std,4.148301,4.169752,6.575378e+05,0.004668
min,0.308055,310.150000,5.132500e+04,0.455896
25%,3.128262,313.038889,5.068806e+05,0.459023
50%,6.675037,316.650000,1.076325e+06,0.462950
75%,10.292277,320.261111,1.645769e+06,0.467018
max,13.550820,323.150000,2.101325e+06,0.471479


And some pretty plots

In [30]:
import plotly.express as px

fig_im = px.imshow(
    df_worst_case.pivot(
    index="T", columns="P", values=deviation_col
    ),
    labels=dict(x="Pressure [Pa]", y="Temperature [K]", color=deviation_col),
    title=f"{deviation_col} vs. Temperature and Pressure",
    aspect="auto",
)
fig_im.update_xaxes(autorange=0)
fig_im.update_yaxes(autorange=0)
fig_im.update_layout(
    template="plotly_dark",
    paper_bgcolor="#1f1f1f",
)
fig_im.show()

fig_hist = px.histogram(
    df_worst_case, x=deviation_col, nbins=20, histnorm="probability density",
    title=f"{deviation_col} Histogram",
)
fig_hist.update_layout(
    template="plotly_dark",
    paper_bgcolor="#1f1f1f",
)
fig_hist.show()